In [1]:
import pandas as pd
import random
import os
import gc

import matplotlib.pyplot as plt

import numpy as np

from bokeh.palettes import Spectral10
from bokeh.io import output_notebook, show

output_notebook()
                                                                                                                                                                                                                                                                                                                                                        
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

palette = Spectral10

pd.set_option('display.float_format', lambda x: '%.3f' % x)

import bokeh
bokeh.__version__

os.chdir("/home/tales/dev/master/mdc_analysis/")
print("working dir", os.getcwd())

from src.dao.csv_dao import load_user_gps_csv, load_user_gps_time_window, load_user_stop_regions_centroids
from src.dao.dbdao import DBDAO
from src.utils.math import normalize
from src.utils.time_utils import enrich_time_columns, local_time
from src.utils import geo
from src.data_processment.stop_region import MovingCentroidStopRegionFinder
from src.plot.plot import add_centroid_figure, add_calculated_centroid_figure
from src.plot import plot
from src.utils.geo import index_clusters
from src.plot import plot
from src.dao import places_dao
from bokeh.layouts import gridplot

Loading BokehJS ...

working dir /home/tales/dev/master/mdc_analysis


In [2]:
def load_users_gps_data(userids, cols=["userid", "latitude", "longitude", "tz", "time", "local_time", "horizontal_accuracy", "horizontal_dop", "speed"]):
    df = pd.DataFrame()
    for userid in userids:
        user_gps_df = load_user_gps_csv(userid)
        user_gps_df["userid"] = [userid] * len(user_gps_df)
        df = df.append(user_gps_df)
        
    df = local_time(df)
    
    if cols != "*":
        df = df[cols]
    
    df = df.sort_values("local_time")

    return df

def load_user_gps_data(userid, cols=["userid", "latitude", "longitude", "tz", "time", "local_time", "horizontal_accuracy", "horizontal_dop", "speed"]):
    user_gps_data = load_user_gps_csv(userid)
    user_gps_data["userid"] = [userid] * len(user_gps_data)
    
    if cols != "*":
        user_gps_data = user_gps_data[cols]
        
    user_gps_data = user_gps_data.sort_values("local_time")
    
    return user_gps_data

In [3]:
user_ids = [6085, 5965] + DBDAO().users_with_places().sample(3).tolist()
user_ids

[6085, 5965, 5985, 6087, 5959]

In [4]:
plot_counter = 1

In [5]:
def plot_visit(visit_locations, color="navy", title="user visits", limit_points=500, p=None, size=8, legend=None, width=800, height=600):
    if len(visit_locations) > limit_points:
        visit_locations = visit_locations.sample(limit_points)
    p = plot.plot_user_loc(user_data=visit_locations[["latitude", "longitude"]].drop_duplicates(), 
                      title=title, width=width, height=height, size=size, color=color, p=p, 
                      legend=legend, alpha=0.3)
    return p

## Match Stop Region with HOME inferred points

In [6]:
def plot_points_and_centroids(points, centroids, user_id, width=600, height=400):
    p = plot.plot_stop_regions_centroids(centroids, title="User: {} - GPS points and Stop Region".format(str(user_id)), width=width, height=height)
    p = plot.plot_user_loc(user_data=points, p=p)
    return p

In [7]:
user_id = 6188
home_points = places_dao.places_home(user_id, do_remove_outliers=True)
home_sr = places_dao.load_stop_regions_home(user_id, verbose=True)["home"]

p = plot_points_and_centroids(home_points, home_sr, user_id, width=600, height=400)
show(p)
print("Plot # {}".format(plot_counter))
plot_counter += 1

821 stop regions

44 stop regions HOME
777 stop regions NOT HOME


Plot # 1


In [8]:
user_id = 6085
home_points = places_dao.places_home(user_id, do_remove_outliers=True)
home_sr = places_dao.load_stop_regions_home(user_id, verbose=True)["home"]

p = plot_points_and_centroids(home_points, home_sr, user_id, width=600, height=400)
show(p)
print("Plot # {}".format(plot_counter))
plot_counter += 1

1398 stop regions

23 stop regions HOME
1375 stop regions NOT HOME


Plot # 2


In [9]:
user_id = 5965
home_points = places_dao.places_home(user_id, do_remove_outliers=True)
home_sr = places_dao.load_stop_regions_home(user_id, verbose=True)["home"]

p = plot_points_and_centroids(home_points, home_sr, user_id, width=600, height=400)
show(p)
print("Plot # {}".format(plot_counter))
plot_counter += 1

1362 stop regions

98 stop regions HOME
1264 stop regions NOT HOME


Plot # 3


#### The home inference can detect multiple places.



In [10]:
user_id = 6086
home_points = places_dao.places_home(user_id, do_remove_outliers=True)
home_sr = places_dao.load_stop_regions_home(user_id, verbose=True)["home"]

p = plot_points_and_centroids(home_points, home_sr, user_id, width=600, height=400)
show(p)
print("Plot # {}".format(plot_counter))
plot_counter += 1

450 stop regions

28 stop regions HOME
422 stop regions NOT HOME


Plot # 4


#### The home inference is robust for moving points stated as home place

## Plotting HOME and NOT HOME Stop Regions

In [11]:
def plot_home_and_others(user_id):
    print("User {}".format(user_id))
    
    sr = home_sr = places_dao.load_stop_regions_home(user_id, verbose=True)
    
    home_sr = sr["home"]
    not_home_sr = sr["not_home"]
        
    p = plot.plot_stop_regions_centroids(not_home_sr, legend="NOT home", fill_color="navy", width=600, height=400)
    p = plot.plot_stop_regions_centroids(home_sr, legend="home", p=p, fill_color="magenta", width=600, height=400)

    return p

#### <font color="blue"> Please apply ZOOM to see Home Stop Regions distribution better </font>

In [12]:
p = plot_home_and_others(6085)
show(p)
print("Plot # {}".format(plot_counter))
plot_counter += 1

User 6085
1398 stop regions

23 stop regions HOME
1375 stop regions NOT HOME


Plot # 5


In [13]:
p = plot_home_and_others(6187)
show(p)
print("Plot # {}".format(plot_counter))
plot_counter += 1

User 6187
187 stop regions

6 stop regions HOME
181 stop regions NOT HOME


Plot # 6


In [14]:
p = plot_home_and_others(6188)
show(p)
print("Plot # {}".format(plot_counter))
plot_counter += 1

User 6188
821 stop regions

44 stop regions HOME
777 stop regions NOT HOME


Plot # 7


In [15]:
p = plot_home_and_others(DBDAO().users_with_places().sample().item())
show(p)
print("Plot # {}".format(plot_counter))
plot_counter += 1

User 5957
508 stop regions

89 stop regions HOME
419 stop regions NOT HOME


Plot # 8


In [16]:
p = plot_home_and_others(6038)
show(p)
print("Plot # {}".format(plot_counter))
plot_counter += 1

User 6038
647 stop regions

3 stop regions HOME
644 stop regions NOT HOME


Plot # 9


<font color="blue"> It seems that there are lots of Stop Regions very close to <b>Home</b> Stop Regions but not marked as Home.
<br>
They were not marked as Home because the time that the person stayed there was not registered in <i>Places</i> table.
<br>
These Stop Regions will be considered Home.
</font>

# Considering Stop Regions close to Home also as Home

In [17]:
def plot_home_expanded_and_others(user_id, radius=20):
    print("User {}".format(user_id))
    sr = places_dao.load_stop_regions_home(user_id, verbose=True)

    home_sr = sr["home"]
    not_home_sr = sr["not_home"]
    
    home_close_sr = geo.infer_close_sr_as_home(home_sr, not_home_sr, radius)

    print(len(home_close_sr), "close to home < {}m".format(radius))
        
    p = plot.plot_stop_regions_centroids(not_home_sr, legend="NOT home", fill_color="navy", width=600, height=400)
    p = plot.plot_stop_regions_centroids(home_close_sr, legend="home close < {}m".format(radius), p=p, fill_color="#ff9a00", width=600, height=400)
    p = plot.plot_stop_regions_centroids(home_sr, legend="home", p=p, fill_color="magenta", width=600, height=400)
     
    return p

#### <font color="blue"> Please apply ZOOM to see Home Stop Regions distribution better </font>

In [18]:
p = plot_home_expanded_and_others(6188)
show(p)
print("Plot # {}".format(plot_counter))
plot_counter += 1

User 6188
821 stop regions

44 stop regions HOME
777 stop regions NOT HOME
66 close to home < 20m


Plot # 10


In [19]:
p = plot_home_expanded_and_others(6038)
show(p)
print("Plot # {}".format(plot_counter))
plot_counter += 1

User 6038
647 stop regions

3 stop regions HOME
644 stop regions NOT HOME
4 close to home < 20m


Plot # 11


In [20]:
p = plot_home_expanded_and_others(6085)
show(p)
print("Plot # {}".format(plot_counter))
plot_counter += 1

User 6085
1398 stop regions

23 stop regions HOME
1375 stop regions NOT HOME
37 close to home < 20m


Plot # 12


In [21]:
p = plot_home_expanded_and_others(DBDAO().users_with_places().sample().item())
show(p)
print("Plot # {}".format(plot_counter))
plot_counter += 1

User 5961
2161 stop regions

642 stop regions HOME
1519 stop regions NOT HOME
1107 close to home < 20m


Plot # 13


In [22]:
p = plot_home_expanded_and_others(DBDAO().users_with_places().sample().item())
show(p)
print("Plot # {}".format(plot_counter))
plot_counter += 1

User 5991
1849 stop regions

15 stop regions HOME
1834 stop regions NOT HOME
18 close to home < 20m


Plot # 14


In [23]:
p = plot_home_expanded_and_others(DBDAO().users_with_places().sample().item())
show(p)
print("Plot # {}".format(plot_counter))
plot_counter += 1

User 5973
6079 stop regions

1695 stop regions HOME
4384 stop regions NOT HOME
3440 close to home < 20m


Plot # 15


In [24]:
p = plot_home_expanded_and_others(DBDAO().users_with_places().sample().item())
show(p)
print("Plot # {}".format(plot_counter))
plot_counter += 1

User 5985
124 stop regions

6 stop regions HOME
118 stop regions NOT HOME
1 close to home < 20m


Plot # 16
